In [ ]:
import random

# Initial setup
stock_price = 100  # Starting price
shares_outstanding = 100000
initial_short = 60000  # HF short position (60% of shares)
float_available = 40000  # Initial float
ri_shares = 20000  # RI initial ownership
ri_cash = 2000000  # RI cash to buy more shares ($2M)

# Player classes
class HedgeFund:
    def __init__(self, short_position, entry_price):
        self.short_position = short_position
        self.entry_price = entry_price
        self.cash = 0  # Gains/losses tracked later

    def decide(self, price, day):
        # Simple strategy: Cover if price spikes too high or hold
        if price > 150 and random.random() < 0.7:  # 70% chance to cover if price > $150
            return "Cover", min(self.short_position, 20000)  # Cover 20k shares max per turn
        return "Hold", 0

    def payoff(self, final_price):
        # Payoff = (entry - final) * shares shorted (negative if price rises)
        return (self.entry_price - final_price) * self.short_position

class RetailInvestors:
    def __init__(self, shares, cash, entry_price):
        self.shares = shares
        self.cash = cash
        self.entry_price = entry_price

    def decide(self, price, float_left, day):
        # Strategy: Buy/Hold early, Sell later if price spikes
        if day < 3 and self.cash > price * 10000:  # Buy up to 10k shares early
            buy_amount = min(10000, int(self.cash / price))
            return "Buy", buy_amount
        elif price > 250 and random.random() < 0.5:  # 50% chance to sell if price > $250
            return "Sell", min(self.shares, 10000)  # Sell 10k shares max
        return "Hold", 0

    def payoff(self, final_price):
        # Payoff = (final - entry) * shares held
        return (final_price - self.entry_price) * self.shares

# Simulate price change
def update_price(price, hf_action, hf_amount, ri_action, ri_amount, float_left):
    demand = 0
    supply = 0
    k = 5  # Sensitivity factor

    if hf_action == "Cover":
        demand += hf_amount
    if ri_action == "Buy":
        demand += ri_amount
    elif ri_action == "Sell":
        supply += ri_amount

    net_change = k * (demand - supply) / float_left  # Price change proportional to float
    new_price = max(10, price * (1 + net_change))  # Minimum price $10
    return new_price

# Main simulation
def run_short_squeeze(days=5):
    hf = HedgeFund(initial_short, stock_price)
    ri = RetailInvestors(ri_shares, ri_cash, stock_price)
    current_price = stock_price
    current_float = float_available

    print(f"Initial Price: ${current_price:.2f}, Float: {current_float}, HF Short: {hf.short_position}, RI Shares: {ri.shares}")

    for day in range(1, days + 1):
        # Players decide
        hf_action, hf_amount = hf.decide(current_price, day)
        ri_action, ri_amount = ri.decide(current_price, current_float, day)

        # Update positions
        if hf_action == "Cover":
            hf.short_position -= hf_amount
            current_float += hf_amount  # Covering returns shares to float
            hf.cash -= hf_amount * current_price  # Cost to buy back
        if ri_action == "Buy":
            ri.shares += ri_amount
            ri.cash -= ri_amount * current_price
            current_float -= ri_amount  # Buying reduces float
        elif ri_action == "Sell":
            ri.shares -= ri_amount
            ri.cash += ri_amount * current_price
            current_float += ri_amount  # Selling increases float

        # Update price
        current_price = update_price(current_price, hf_action, hf_amount, ri_action, ri_amount, current_float)
        current_float = max(1000, current_float)  # Minimum float

        print(f"Day {day}: Price=${current_price:.2f}, Float={current_float}, HF={hf_action} ({hf_amount}), RI={ri_action} ({ri_amount})")

    # Final payoffs
    hf_payoff = hf.payoff(current_price)
    ri_payoff = ri.payoff(current_price)
    print(f"\nFinal Price: ${current_price:.2f}")
    print(f"HF Payoff: ${hf_payoff / 1000000:.2f}M (Short remaining: {hf.short_position})")
    print(f"RI Payoff: ${ri_payoff / 1000000:.2f}M (Shares held: {ri.shares})")

# Run the simulation
random.seed(42)  # For reproducibility
run_short_squeeze()

Initial Price: $100.00, Float: 40000, HF Short: 60000, RI Shares: 20000
Day 1: Price=$266.67, Float=30000, HF=Hold (0), RI=Buy (10000)
Day 2: Price=$488.89, Float=60000, HF=Cover (20000), RI=Sell (10000)
Day 3: Price=$760.49, Float=90000, HF=Cover (20000), RI=Sell (10000)
Day 4: Price=$760.49, Float=90000, HF=Hold (0), RI=Hold (0)
Day 5: Price=$380.25, Float=100000, HF=Hold (0), RI=Sell (10000)

Final Price: $380.25
HF Payoff: $-5.60M (Short remaining: 20000)
RI Payoff: $0.00M (Shares held: 0)
